In [1]:
import pandas as pd

# Charger les données
df = pd.read_csv("full_grouped.csv")  

# Renommer les colonnes pour simplifier
df = df.rename(columns={
    'Date': 'Date',
    'Country/Region': 'Country',
    'Confirmed': 'Confirmed',
    'Deaths': 'Deaths',
    'Recovered': 'Recovered',
    'New cases': 'New cases',
    'New deaths': 'New deaths',
    'New recovered': 'New recovered'
})

# Convertir la date et créer une colonne mois
df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.to_period('M').dt.to_timestamp()

# Garder uniquement les colonnes nécessaires
df = df[['Month', 'Country', 'Confirmed', 'Deaths', 'Recovered',
         'New cases', 'New deaths', 'New recovered']]

# Convertir les colonnes temporelles en valeurs absolues
for col in ['New cases', 'New deaths', 'New recovered']:
    df[col] = df[col].abs()

# Grouper par mois et pays
df_grouped = df.groupby(['Month', 'Country'], as_index=False).agg({
    'Confirmed': 'last',
    'Deaths': 'last',
    'Recovered': 'last',
    'New cases': 'sum',
    'New deaths': 'sum',
    'New recovered': 'sum'
})

# Ajouter une colonne Id auto-incrémentée
df_grouped.insert(0, 'Id', range(1, len(df_grouped) + 1))

# Sauvegarder le résultat dans le répertoire courant
df_grouped.to_csv("data_etl_output.csv", index=False)

print("ETL terminé. Fichier sauvegardé sous : data_etl_output.csv")


ETL terminé. Fichier sauvegardé sous : data_etl_output.csv
